In [2]:
import random
import ast
import pandas as pd
import numpy as np
import re

## Load Club CSV

In [3]:
data = pd.read_csv('clubs_200_withviewcnt.csv') 
data = data.iloc[:,1:]
data.head() #추후 tag 예측한거까지 다 하면 200개 대신 전체로 변경

,0,1,2,3,4,5,clubName,categories,viewed
0,401,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다!\n\r\n?수레바...,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,수레바퀴,"['학회', '경영']",74
1,402,[2020 YOUNG TIGERS 신입단원 모집 D-day],https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/de478d7/resize'],[2020 YOUNG TIGERS 신입단원 모집]\n\r\n안녕하십니까 고려대학교 ...,2020 YOUNG TIGERS 신입단원 모집 Dday 2020 YOUNG TIGE...,Young Tigers,"['운동', '동아리']",108
2,403,??[KULSOM D-1] 마감 하루 전!!??????,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/6f7ae51/resize'],??지원 마감 하루 전입니다! \n\r\n??한 학기 같이 즐거운 추억 만들어요! ...,KULSOM D1 마감 하루 전!! 지원 마감 하루 전입니다! 한 학기 같이 즐...,KULSOM,"['학회', '경영']",98
3,404,[柔道部] 유도부 상시모집 ??,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/5b0b384/resize', ...",??고대 유도부 신입부원 상시모집??\r\n????아무나 와라 강하게 키워줄게???...,유도부 상시모집 고대 유도부 신입부원 상시모집 아무나 와라 강하게 키워줄게 고...,유도부,"['동아리', '운동']",86
4,405,[D-3]??KU-HOPE 2020년 1학기 리크루팅!??,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/e01d297/resize'],??재난에서 희망으로! 안녕하세요 고려대학교 집수리봉사동아리 KU-HOPE 입니다....,D3KUHOPE 2020년 1학기 리크루팅! 재난에서 희망으로! 안녕하세요 고려대학...,KU-HOPE,"['봉사', '동아리']",119


In [4]:
data.categories = data.categories.apply(ast.literal_eval)

In [5]:
alll = []
for _ in data.categories:
    alll+=_
alll = list(set(alll)) #모집?비거니즘??
print(alll)

['대외활동', '언어', '모집', '학회', '종교', '노래', '스터디', '봉사', '동아리', '운동', '경영', '사회', '예술', '음악', '연합', '친목', 'IT', '비거니즘']


### [For CBF] Preprocess (Insertion, Deletion에 대한 TFIDF계산 해결해야 함)
하루에 한번씩 업데이트 하는 코드 따로 생성

In [6]:
clubstxt = data.iloc[:,5] #특수문자 제거 되어있음
clubstxt

0      중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...
1      2020 YOUNG TIGERS 신입단원 모집 Dday 2020 YOUNG TIGE...
2      KULSOM D1 마감 하루 전!! 지원 마감 하루 전입니다!   한 학기 같이 즐...
3       유도부 상시모집  고대 유도부 신입부원 상시모집 아무나 와라 강하게 키워줄게  고...
4      D3KUHOPE 2020년 1학기 리크루팅! 재난에서 희망으로! 안녕하세요 고려대학...
                             ...                        
195    고려대 유일 사격동아리 '명중사격부' 신입부원 모집 너의 마음을 명중 넌 이미 신청...
196     중앙동아리 예술비평연구회에서 신입회원을 모집합니다!   2020년 1학기는 세미나...
197    오늘 8시 마감고려대학교 유일의 브랜드 컨설팅 학회 CREAKET에서 14기를 모집...
198    D1 Partyproviders 27th 2차 리크루팅 3.22 많은 문의와 요청에...
199    KUHS 고려대학교 호스피스 봉사동아리 (KUHS)에서 신입부원을 모집합니다! (일...
Name: 5, Length: 200, dtype: object

In [7]:
#불용어 불러오기
stopwords = pd.read_pickle('stopwords.pkl')
stopwords = list(stopwords)

#tokenizing
from konlpy.tag import Komoran
komo = Komoran() #어미/어근의 분류가 잘 되어있다 판단.

class Tokenize:
    def __init__(self):
        pass
    def __call__(self, text):
        #stemming
        #어미 관련 모두 제외
        stemmed = [_ for _ in komo.pos(text) if _[1] not in ['EC', 'EF', 'EP', 'ETM', 'ETN']]
        
        #lemmatizing
        #VV=동사어근, VA=형용사어근
        lemma = [_[0]+'다' if (_[1]=='VA') or (_[1]=='VV') else _[0] for _ in stemmed]
        
        #remove stopwords
        tokens = [_ for _ in lemma if _ not in stopwords]
        
        return tokens
    

In [8]:
tokenize = Tokenize()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

#ngram, stopwords 기능 지원
#객체 생성
tfidf = TfidfVectorizer(tokenizer = tokenize,
                        analyzer = 'word', ngram_range=(1,2), 
                        min_df=0) #ngram called here

In [10]:
tfidf_matrix = tfidf.fit_transform(clubstxt)

In [11]:
tfidf_matrix.shape

(200, 24658)

In [12]:
from sklearn.metrics.pairwise import linear_kernel
#linear_kernel = dot product
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

sorted(cosine_sim[0],reverse=True)[1:10]

[0.9885866433178748,
 0.946963656606428,
 0.09408093366423495,
 0.08573957388650287,
 0.08434785808134902,
 0.08372156296017692,
 0.08368143958290032,
 0.08275697907426006,
 0.08253045494638345]

### 과제 : cosine similarity matrix 저장하기 (업데이트 코드에)

In [13]:
def get_recommendations(title):
    #index에대한 제목
    titles = data.iloc[:,1]
    
    #제목에 대한 index
    indices = pd.Series(data.index, index=data.iloc[:,1])
    i = indices[title]
    
    #입력한 동아리에대한 cosine_sim가져오기
    sim_scores = list(enumerate(cosine_sim[i]))
    
    #정렬
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)[1:21]

    return titles[[s[0] for s in sim_scores]]

In [15]:
titles = data.iloc[:,1]
titles[1]

'[2020 YOUNG TIGERS 신입단원 모집 D-day] '

In [16]:
get_recommendations(titles[1])

35                          [2020 YOUNG TIGERS 신입단원 모집] 
86                          [2020 YOUNG TIGERS 신입단원 모집] 
123                         [2020 YOUNG TIGERS 신입단원 모집] 
157                       [2020 YOUNG TIGERS 신입단원 모집] ? 
191                         [2020 YOUNG TIGERS 신입단원 모집] 
60                        [2020 YOUNG TIGERS 신입단원 모집] ? 
26                                      고려대학교 새내기맞이단 모집 
12                  [국별 활동 업로드] ?? 고려대학교 새내기맞이단 4기 모집?? 
30                           ♧고려대학교 합창단♧에서 신입단원을 모집합니다! 
160                               ??고려대학교 합창단?? 신입단원 모집 
54                                  ☆고려대학교 합창단☆ 신입단원 모집 
89                                ??고려대학교 합창단 신입단원 모집?? 
18                         ?? 2020 TIGERS Recruiting ?? 
151         [비대면면접/라디오드라마] 고려대학교 성우동아리 온보이싱 26기 신입부원 모집 
168    [KUHS] 고려대학교 ‘호스피스 봉사동아리 (KUHS)’에서 신입부원을 모집합니다...
199    [KUHS] 고려대학교 ‘호스피스 봉사동아리 (KUHS)’에서 신입부원을 모집합니다...
44     [KUHS] 고려대학교 ‘호스피스 봉사동아리 (KUHS)’에서 신입부원을 모집합니다...
102    [KUHS] 고려대학교 ‘호스피스 봉사동아리

## Create User Table (Temporary)

In [73]:
usertable = pd.DataFrame(columns = ['id','user_id','grade','sex','college','total_view'])

In [74]:
grade = ['1학년','2학년','3학년','4학년']  
sex = ['M','F']
college = ['정보대학', '정보보호학부','공과대학','이과대학','정경대학','경영대학','간호대학','보건과학대학','국제학부','디자인조형학부','미디어학부','문과대학','사범대학','생명과학대학','의과대학']

In [75]:
for i in range(100):
    tmp = [i+1]
    tmp.append(i+1) #나중에는 유저아이디
    tmp.append(random.choice(grade))
    tmp.append(random.choice(sex))
    tmp.append(random.choice(college))
    tmp.append(random.randint(0,40))
    usertable.loc[i]=tmp

In [76]:
usertable

,id,user_id,grade,sex,college,total_view
0,1,1,1학년,M,사범대학,34
1,2,2,4학년,M,공과대학,13
2,3,3,2학년,M,생명과학대학,1
3,4,4,3학년,M,국제학부,35
4,5,5,4학년,F,의과대학,26
...,...,...,...,...,...,...
95,96,96,1학년,F,미디어학부,32
96,97,97,1학년,M,간호대학,12
97,98,98,2학년,F,간호대학,35
98,99,99,2학년,F,사범대학,27


In [77]:
i = usertable.loc[usertable.user_id==1]
i.grade[0]

'1학년'

## 조회테이블

In [116]:
def simulation(userId):
    #userId 가 봤을법한 동아리들을 simpleRS에서 가져온다
    candi = simpleRS(userId)
    
    #usertable 내 조회수 만큼, simpleRS토대로 동아리 랜덤 조회 pick
    selected = pd.Series(np.random.choice(candi,usertable.loc[userId-1]['total_view']))
    
    #홍보물에 대한 index값
    indices = pd.Series(data.index, index=data.iloc[:,-3])
    
    df = pd.DataFrame(selected.apply(lambda x : indices[x][0] if type(indices[x])==pd.core.series.Series else indices[x]),columns=['clubId'])
    df['userId'] = userId
    df['viewed']= 1
    
    return df

In [117]:
viewtable = pd.DataFrame(columns = ['clubId','userId','viewed'])
for i in range(100):
    viewtable=viewtable.append(simulation(i+1),ignore_index=True)
viewtable

<학년:1학년, 성별:M, 단과대학:사범대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:4학년, 성별:M, 단과대학:공과대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:2학년, 성별:M, 단과대학:생명과학대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:3학년, 성별:M, 단과대학:국제학부> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:4학년, 성별:F, 단과대학:의과대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:4학년, 성별:F, 단과대학:정경대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:3학년, 성별:M, 단과대학:간호대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:4학년, 성별:M, 단과대학:디자인조형학부> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:1학년, 성별:F, 단과대학:디자인조형학부> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:4학년, 성별:M, 단과대학:정보보호학부> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:2학년, 성별:F, 단과대학:공과대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:4학년, 성별:F, 단과대학:생명과학대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:3학년, 성별:M, 단과대학:정보대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:1학년, 성별:M, 단과대학:디자인조형학부> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:3학년, 성별:F, 단과대학:이과대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:4학년, 성별:F, 단과대학:정보보호학부> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:3학년, 성별:F, 단과대학:디자인조형학부> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열
<학년:1학년, 성별

,clubId,userId,viewed
0,12,1,1
1,18,1,1
2,33,1,1
3,5,1,1
4,155,1,1
...,...,...,...
2071,32,100,1
2072,52,100,1
2073,39,100,1
2074,17,100,1


## Simple RS

In [113]:
def simpleRS(user_id):
    
    graded = {'1학년': ['동아리', '친목', '연합'],
                '2학년' : ['동아리','대외활동','봉사'],
                '3학년' : ['대외활동', '봉사'],
                '4학년' : ['학회', '스터디', '공모전']}


    sexd = {'F' : ['음악','예술', '언어'],
            'M' : ['운동', '음악', 'IT']}


    colleged = {'정보대학' : ['IT'],
                '정보보호학부' : ['IT'],
                '공과대학' : ['IT'],
                '이과대학' : ['IT'],
                '정경대학' :  ['경영', '대외활동'], 
                '경영대학' :  ['경영', '대외활동'], 
                '간호대학':['사회'], 
                '보건과학대학':['사회'],
                '국제학부' : ['언어'],
                '디자인조형학부' : ['예술'],
                '미디어학부' : ['경영', '대외활동', '예술', '음악'],
                '문과대학' : ['대외활동', '사회', '언어'] , 
                '사범대학' : ['대외활동', '사회', '언어'] ,
                '생명과학대학':['봉사'], 
                '의과대학' : ['봉사']}

    userinfo = usertable.loc[usertable.user_id==user_id]
    
    selected = list(set(graded[userinfo.grade.iloc[0]]+sexd[userinfo.sex.iloc[0]]+colleged[userinfo.college.iloc[0]]))
    
    ## 사용자정보에 매핑하여 속하는 동아리 추출
    data_selected = data.loc[data.categories.apply(lambda y : any(x in y for x in selected))]
    
    ## 조회수별로 정렬
    selected_ordered = data_selected.sort_values(by='viewed', ascending=False)

    print('<학년:{}, 성별:{}, 단과대학:{}> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열'.format(
        userinfo.grade.iloc[0],userinfo.sex.iloc[0],userinfo.college.iloc[0]))
    return selected_ordered.clubName.reset_index().clubName

In [23]:
simpleRS(1)

<학년:1학년, 성별:M, 단과대학:경영대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열


0      Young Tigers
1               엔더스
2               코기토
3           예술비평연구회
4              고란도란
           ...     
173            고란도란
174          KULSOM
175        soulmate
176           성우동아리
177             엔더스
Name: clubName, Length: 178, dtype: object

## CBF
조회했던 홍보물을 모두 모아 각 홍보물-나머지 홍보물들에 대한 cosine similarity를 구한다. 그 중 가장 cosine similarity가 높은것부터 나열

In [280]:
#viewtable
def cbf(userId,no_recom=10):
    #index에대한 제목
    names = data.iloc[:,-3]
    
    #조회한 동아리들에대한 cosine_sim 모두 더하기(조회수 2회이상이면 그만큼 곱하기)
    sim_scores=np.zeros(len(cosine_sim[0]))
    for n,i in enumerate(viewtable[viewtable['userId']==userId]['clubId']):
        sim_scores += (np.array(cosine_sim[i])*viewtable[viewtable['userId']==userId].reset_index()['viewed'][n])
    
    sim_scores = list(enumerate(sim_scores))
    #sim 높은순부터 정렬
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)
    res = names[[s[0] for s in sim_scores]]
    tmp = []
    i=0
    while(len(tmp)<no_recom):
        if res.iloc[i] not in tmp:
            tmp.append(res.loc[i])
        i+=1
    
    return tmp

In [281]:
cbf(2)

['수레바퀴',
 'Young Tigers',
 'KULSOM',
 '유도부',
 'KU-HOPE',
 '고란도란',
 '고집',
 '고란도란',
 '맥락',
 '쿠리에이터']

# CF(Matrix Factorization)

In [180]:
clubtable = pd.DataFrame({'clubId':pd.Series(range(200)), 'clubName':data['clubName']})
clubtable

,clubId,clubName
0,0,수레바퀴
1,1,Young Tigers
2,2,KULSOM
3,3,유도부
4,4,KU-HOPE
...,...,...
195,195,명중
196,196,예술비평연구회
197,197,CREAKET
198,198,Partyproviders


In [231]:
user_club = viewtable.merge(clubtable, how='right', on='clubId')
user_club['viewed'] = pd.to_numeric(user_club['viewed'])
user_club['clubId'] = user_club['clubId'].apply(lambda x: int(x) if not pd.isna(x) else x)
user_club

,clubId,userId,viewed,clubName
0,0,2,1.0,수레바퀴
1,0,5,1.0,수레바퀴
2,0,10,1.0,수레바퀴
3,0,10,1.0,수레바퀴
4,0,12,1.0,수레바퀴
...,...,...,...,...
2178,198,67,1.0,Partyproviders
2179,198,78,1.0,Partyproviders
2180,198,81,1.0,Partyproviders
2181,198,90,1.0,Partyproviders


In [233]:
user_item = user_club.pivot_table(values = 'viewed', index = 'userId', columns = 'clubId',dropna=False)
user_item.head()

clubId,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [237]:
#nan값 대신 0 으로 채우기
user_item=user_item.fillna(0)
user_item.columns = data['clubName']
user_item

clubName,수레바퀴,Young Tigers,KULSOM,유도부,KU-HOPE,고란도란,고집,고란도란,맥락,쿠리에이터,...,KU-HOPE,Young Tigers,KULSOM,Clo.Z,로그스,명중,예술비평연구회,CREAKET,Partyproviders,KUHS
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
97,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [238]:
from numpy.linalg import svd
from scipy.sparse.linalg import svds #does truncated svd

U, sigma, Vt = svd(user_item)
#find cumsum
cumratio = list(map(lambda x : round(x,3), np.cumsum(sigma)/np.sum(sigma)))

#80%의 설명력을 가지고 있는 요소들만 pick
toleaveout = len(list(filter(lambda x : x<0.8, cumratio)))

U, sigma, Vt = svds(user_item, k = toleaveout) #k=남길 특이값 개수

sigma = np.diag(sigma)

#convert back to original matrix by matrix multiplaction
svd_pred = np.dot(np.dot(U,sigma),Vt)
svd_pred = pd.DataFrame(svd_pred)
svd_pred #값이 달라짐 - truncated를 썼기 때문

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.060595,-0.093924,0.043867,0.059064,0.042211,0.840105,0.088330,0.0,0.008529,1.055111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.054938,0.011015,0.073027,0.0
1,0.961887,0.094623,0.028187,-0.026451,-0.058470,0.050767,0.089985,0.0,0.072611,0.886232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.146772,-0.014205,0.016271,0.0
2,-0.008381,0.053220,0.021403,-0.067778,-0.014809,0.072791,-0.112650,0.0,-0.115216,0.032732,...,0.0,0.0,0.0,0.0,0.0,0.0,0.012045,-0.025749,-0.018497,0.0
3,0.102145,1.073487,0.966529,0.979610,-0.039290,0.940106,1.028030,0.0,0.132743,0.008856,...,0.0,0.0,0.0,0.0,0.0,0.0,0.025134,-0.076505,0.201012,0.0
4,0.951156,-0.074512,0.976937,0.110752,0.846569,0.945407,0.049516,0.0,-0.088928,-0.138183,...,0.0,0.0,0.0,0.0,0.0,0.0,0.965139,0.173549,-0.028463,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1.009926,-0.078727,0.951093,-0.004273,0.090167,0.975815,-0.034693,0.0,0.888336,1.051678,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.145536,0.849763,0.100016,0.0
93,0.043130,0.071405,0.031481,0.919652,0.094108,0.987619,0.073428,0.0,0.058069,0.872622,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.080497,0.186348,0.118789,0.0
94,-0.054709,1.159744,-0.058841,-0.065153,0.041620,0.982924,0.975352,0.0,0.063442,0.929518,...,0.0,0.0,0.0,0.0,0.0,0.0,0.025922,-0.052708,0.043573,0.0
95,0.018739,0.948163,0.924873,-0.010244,0.014374,0.903520,0.125431,0.0,0.979609,-0.018541,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.031175,0.119203,0.070928,0.0


In [282]:
def cf(user_id, num_recom=10):
    index = user_id-1
    
    #sort expected ratings
    sorted_index = svd_pred.loc[user_id].sort_values(ascending=False).index
    
    #get name list of clubs seen by user
    seen_clubs = svd_pred.columns[user_item.loc[user_id]!=0]
    
    #get index list of clubs seen by user
    seen_names = user_item.loc[user_id][user_item.loc[user_id]!=0].index
    seen_names = list(seen_names.unique())

    #get top n from movies user has not seen
    sorted_notseen = [x for x in sorted_index if x not in seen_clubs]
    recom_title = list(user_item.columns[sorted_notseen][:num_recom])
    
    print('what you have seen : {}'.format(seen_names))
    print('Recommended for you : {}'.format(recom_title))
    return recom_title

In [283]:
cf(50)

what you have seen : ['쿠리에이터', '엔더스', '고대농악대', 'KUECS', 'HoKU', 'Clo.z', '예수전도단', '청년성서모임', 'KU HOPE', 'EBS on Campus']
Recommended for you : ['Young Tigers', 'KULSOM', 'KUHS', '로그스', '불아스', '새내기맞이단', '유도부', '와일드아이즈', 'KUBT', 'Talk Through Piano']


['Young Tigers',
 'KULSOM',
 'KUHS',
 '로그스',
 '불아스',
 '새내기맞이단',
 '유도부',
 '와일드아이즈',
 'KUBT',
 'Talk Through Piano']

# 종합

#### USER PHASE
phase1) 초기상태(조회 10 미만)  
- simple recommender*1  
  
phase2) 조회 10 이상  
- simple*0.1 + content based*0.6 + collaborative*0.3  
  
phase3) 조회 20 이상  
- simple*0.1 + content based*0.4 + collaborative*0.5  


In [284]:
simpleRS(3)

<학년:2학년, 성별:M, 단과대학:생명과학대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열


0      Young Tigers
1               엔더스
2               코기토
3             쿠리에이터
4            새내기맞이단
           ...     
151            고란도란
152          KULSOM
153        soulmate
154           성우동아리
155             엔더스
Name: clubName, Length: 156, dtype: object

In [285]:
cbf(3)

['수레바퀴',
 'Young Tigers',
 'KULSOM',
 '유도부',
 'KU-HOPE',
 '고란도란',
 '새내기맞이단',
 'SBC',
 '형사판례연구반',
 '엔더스']

In [286]:
cf(3)

what you have seen : ['Debate Club']
Recommended for you : ['soulmate', 'KUHS', 'Young Tigers', 'SPORTS KU', '엔더스', '호박회', '고집', '코기토', '산악회', '유도부']


['soulmate',
 'KUHS',
 'Young Tigers',
 'SPORTS KU',
 '엔더스',
 '호박회',
 '고집',
 '코기토',
 '산악회',
 '유도부']

In [328]:
def recommendation_system(user_id):
    userinfo = usertable.loc[usertable.user_id==user_id]
    notfirsttime=True
    
    if userinfo.total_view.iloc[0]<10:
        notfirsttime=False
        ratio = [1,0,0]
        
    elif userinfo.total_view.iloc[0]<20: ratio=[0.1,0.6,0.3]
    else: ratio=[0.1,0.3,0.6]
    
    
    result = dict()

    #simpleRS
    SRSres = simpleRS(user_id)
    for n,_ in enumerate(SRSres):
        if _ not in result.keys():
            result[_]=ratio[0]*(abs(n-len(SRSres))/len(SRSres)) #minmaxscalar inverted (ranking에 점수주기)
            
    #when 조회 less than 10, do not apply cbf and cf
    if notfirsttime:
        #cbf
        for _ in cbf(user_id):
            if _ in result.keys():
                result[_]+=ratio[1]
            else:
                result[_]=ratio[1]

        #cf
        for _ in cf(user_id):
            if _ in result.keys():
                result[_]+=ratio[2]
            else:
                result[_]=ratio[2]

    #sort by weight
    result = sorted(result.items(), key=lambda x: x[1],reverse=True)
    return pd.Series([_[0] for _ in result])

In [329]:
recommendation_system(3)

<학년:2학년, 성별:M, 단과대학:생명과학대학> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열


0     Young Tigers
1              엔더스
2              코기토
3            쿠리에이터
4           새내기맞이단
          ...     
72              돌빛
73             서우회
74         학생복지위원회
75        soulmate
76           성우동아리
Length: 77, dtype: object